# **Convolution Neural Network**


---


- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.


---

- **(코랩에서)한글폰트 설치하기**

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 런타입 > 세션 다시 시작

In [ ]:
import matplotlib.pyplot as plt

# 한글 폰트 경로 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
# matplotlib에서 기본 폰트로 지정
plt.rcParams['font.family'] = 'NanumGothic'



---



## **주요 개념**

### **CNN(Convolution Neural Network)**
- **CNN은 이미지나 영상 데이터를 처리하기 위해 설계된 딥러닝 신경망 구조**
- 이미지 속 특징(선, 모서리, 곡선, 패턴) 을 자동으로 추출
- 컴퓨터 비전, 자율주행, 의료 영상 등 다양한 분야에 사용
- Convolutional neural network는 수십 또는 수백 개의 계층을 가질 수 있으며, 각 계층은 영상의 서로 다른 특징을 검출함
- 각 훈련 영상에 서로 다른 해상도의 필터가 적용되고, 컨벌루션된 각 영상은 다음 계층의 입력으로 사용됨
- 필터(커널)는 밝기, 경계와 같이 매우 간단한 특징으로 시작하여 객체를 고유하게 정의하는 특징으로 복잡도를 늘려갈 수 있음

### **커널(Kernel)**
- 커널(kernel)은 필터(filter), 마스크(mask)와 같은 말
- **커널의 기능**은 **이미지로부터 내가 원하는 정보(특징)만을 추출하는 것**
- **커널**은 **이미지 데이터를 지정된 간격(stride)으로 움직이며 합성곱(convolution)을 수행**
- 합성곱 연산을 통해 나온 결과가 **feature map**
- 커널은 이미지의 특징을 찾아내기 위한 공용 파라미터임
- 일반적으로 (2,2), (3,3) 등의 정사각 행렬으로 정의됨
- CNN에서 학습의 대상은 필터 파라미터 임
- 입력 데이터의 각각의 채널에서 지정된 간격으로 순회하며 합성곱을 하고 모든 채널의 합성곱의 합인 feature map이 출력으로 나옴



---



## **합성곱 연산**(Convolution 연산)

### **벡터와 행렬**

In [ ]:
import numpy as np

# numpy에서 행벡터처럼 출력하기 = 1차 행렬
data = [1,2,3]
data1= np.array(data)
print(data1)
print(data1.shape)

### **브로드 캐스트(broadcast)**

In [ ]:
import numpy as np

data = np.array([[1,2],[3,4]])
print(data * 10)
print('-'*30)

data1 = np.array([[1,2],[3,4]])
data2 = np.array([10,20])
print(data1 * data2)

### **벡터의 내적**
내적' 연산이 바로 CNN의 원리를 이루는 가장 핵심적인 연산

In [ ]:
data1 = np.array([1,2,3])
data2 = np.array([4,5,6])
print(data1 @ data2)            # 벡터의 내적 : 특징 맵 (Feature Map)의 1개 픽셀 값
print( np.dot(data1, data2) )
print('-'*30)

data1 = np.array([1,2])
data2 = np.array([[1,1,1],[2,2,2]])
print(data1 @ data2)
print('-'*30)

data1 = np.array([[1,1,1],[2,2,2]])
data2 = np.array([1,2,3])
print(data1 @ data2)

### **[실습]  Numpy를 이용한 합성곱 연산**

In [ ]:
import numpy as np

def getFeatureMap(data, mask):
    result = []
    x, y   = np.shape(data)
    fx, fy = np.shape(mask)
    for i in range(x-fx+1):
        for j in range(y-fy+1):
            result.append((data[i:i+fy, j:j+fy] * mask).sum())

    return np.array(result).reshape(2,2)


# input array
data = np.array([[1,2,2,0],
              [0,1,2,3],
              [1,0,1,2],
              [2,3,0,1]])
# convolution kernel(==filter,mask)
mask = np.array([[0,0,0],
              [0,1,0],
              [0,0,0]])
print(f'#input :\n{data}')
print(f'#convolution kernel:\n{mask}')

# Feature Map(합성곱결과) 만들기
result = getFeatureMap(data, mask)
print(f'#feature map:\n{result}')

# Feature Map에서 map pooling 값 없기
print(f'#max pooling:\n{np.max(result)}')



---



## **Convolution 연산 실습**


### **필터 1개 사용**

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as img

fig = plt.figure(figsize=(10,10))  # 가로세로 크기 inch단위

ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

ori_image = img.imread('/content/image.png')
ax1.imshow(ori_image)

print(ori_image.shape)
# 원본 이미지의 shape : (429, 640, 3) => (height, width, color)

In [ ]:
# 입력이미지의 형태
# (1, 429, 640, 3) => (이미지 개수, height, width, color)
input_image = ori_image.reshape((1,) + ori_image.shape)
input_image = input_image.astype(np.float32)
print('Convolution input image.shape : {}'.format(input_image.shape))

In [ ]:
# 입력이미지 channel 변경
# (1, 429, 640, 1) => (이미지 개수, height, width, color)
# slicing을 이용하여 첫번째 R(Red) 값만 이용
channel_1_input_image = input_image[:,:,:,0:1]
print('Channel 변경 input_image.shape : {}'.format(channel_1_input_image.shape))

In [ ]:
# filter
# (3,3,1,1) => (filter height, filter width, filter channel, filter 개수)
# weight = np.random.rand(3,3,1,1)
weight = np.array([[[[-1]],[[0]],[[1]]],
                   [[[-1]],[[0]],[[1]]],
                   [[[-1]],[[0]],[[1]]]])
print('적용할 filter shape : {}'.format(weight.shape))

In [ ]:
# stride : 1 (가로1, 세로1)
# padding = 'VALID'

conv2d = tf.nn.conv2d(channel_1_input_image,
                      weight,
                      strides=[1,1,1,1],
                      padding='VALID')

conv2d_result = conv2d.numpy()

print('Convolution 결과 shape : {}'.format(conv2d_result.shape))


In [ ]:
# 아래의 코드는 이미지가 1장이기 때문에 필요없다.
# i = np.swapaxes(conv2d_result,0,3)    # (1, 424, 638, 1)
#                                       # (이미지개수, height, weight, filter 수)
#                                       # filter수만큼 loop 돌리기 위해 axes swap

# for filter_idx, t_img in enumerate(i):
#     ax2.imshow(t_img)

ax1.imshow(ori_image)

t_img = conv2d_result[0,:,:,:]
ax2.imshow(t_img, cmap='gray')



fig.tight_layout()
plt.show()

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as img

fig = plt.figure(figsize=(10,10))  # 가로세로 크기 inch단위

ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

ori_image = img.imread('/content/image.png')
ax1.imshow(ori_image)

# 원본 이미지의 shape : (429, 640, 3) => (height, width, color)

# 입력이미지의 형태
# (1, 429, 640, 3) => (이미지 개수, height, width, color)
input_image = ori_image.reshape((1,) + ori_image.shape)
input_image = input_image.astype(np.float32)
print('Convolution input image.shape : {}'.format(input_image.shape))

# 입력이미지 channel 변경
# (1, 429, 640, 1) => (이미지 개수, height, width, color)
# slicing을 이용하여 첫번째 R(Red) 값만 이용
channel_1_input_image = input_image[:,:,:,0:1]
print('Channel 변경 input_image.shape : {}'.format(channel_1_input_image.shape))


# filter
# (3,3,1,1) => (filter height, filter width, filter channel, filter 개수)
# weight = np.random.rand(3,3,1,1)
weight = np.array([[[[-1]],[[0]],[[1]]],
                   [[[-1]],[[0]],[[1]]],
                   [[[-1]],[[0]],[[1]]]])
# weight = np.array([[[[-2]],[[0]],[[1]]],
#                    [[[-2]],[[0]],[[1]]],
#                    [[[-2]],[[0]],[[1]]]])
print('적용할 filter shape : {}'.format(weight.shape))

# stride : 1 (가로1, 세로1)
# padding = 'VALID'

conv2d = tf.nn.conv2d(channel_1_input_image,
                      weight,
                      strides=[1,1,1,1],
                      padding='VALID')

conv2d_result = conv2d.numpy()

print('Convolution 결과 shape : {}'.format(conv2d_result.shape))

# 아래의 코드는 이미지가 1장이기 때문에 필요없다.
# i = np.swapaxes(conv2d_result,0,3)    # (1, 424, 638, 1)
#                                       # (이미지개수, height, weight, filter 수)
#                                       # filter수만큼 loop 돌리기 위해 axes swap

# for filter_idx, t_img in enumerate(i):
#     ax2.imshow(t_img)

t_img = conv2d_result[0,:,:,:]
ax2.imshow(t_img, cmap='gray')

fig.tight_layout()
plt.show()

### **필터 여러 개 사용** : (3개)

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as img

fig = plt.figure(figsize=(10,10))  # 가로세로 크기 inch단위

ax1 = fig.add_subplot(1,4,1)
ax2 = fig.add_subplot(1,4,2)
ax3 = fig.add_subplot(1,4,3)
ax4 = fig.add_subplot(1,4,4)

ori_image = img.imread('./image.png')
ax1.imshow(ori_image)

# 원본 이미지의 shape : (429, 640, 3) => (height, width, color)

# 입력이미지의 형태
# (1, 429, 640, 3) => (이미지 개수, height, width, color)
input_image = ori_image.reshape((1,) + ori_image.shape)
input_image = input_image.astype(np.float32)
print('Convolution input image.shape : {}'.format(input_image.shape))

# 입력이미지 channel 변경
# (1, 429, 640, 1) => (이미지 개수, height, width, color)
# slicing을 이용하여 첫번째 R(Red) 값만 이용
channel_1_input_image = input_image[:,:,:,0:1]
print('Channel 변경 input_image.shape : {}'.format(channel_1_input_image.shape))


# filter
# (3,3,1,3) => (filter height, filter width, filter channel, filter 개수)
# weight = np.random.rand(3,3,1,3)
weight = np.array([[[[-1,-1,-1]],[[0,0,-2]],[[1,1,-1]]],
                   [[[-1,-2,0]],[[0,0,0]],[[1,2,0]]],
                   [[[-1,-1,1]],[[0,0,2]],[[1,1,1]]]])

print('적용할 filter shape : {}'.format(weight.shape))

# 수직선을 강조하는 필터
# [[-1, 0, 1],
#  [-2, 0, 2]
#  [-1, 0, 1]]

# 수평선을 강조하는 필터
# [[-1, -2, -1],
#  [0, 0, 0]
#  [1, 2, 1]]


# stride : 1 (가로1, 세로1)
# padding = 'VALID'

conv2d = tf.nn.conv2d(channel_1_input_image,
                      weight,
                      strides=[1,1,1,1],
                      padding='VALID')

conv2d_result = conv2d.numpy()

print('Convolution 결과 shape : {}'.format(conv2d_result.shape))

# 아래의 코드는 이미지가 1장이기 때문에 필요없다.
# i = np.swapaxes(conv2d_result,0,3)    # (1, 424, 638, 3)
#                                       # (이미지개수, height, weight, filter 수)
#                                       # filter수만큼 loop 돌리기 위해 axes swap

# for filter_idx, t_img in enumerate(i):
#     ax2.imshow(t_img)

ax2.imshow(conv2d_result[0,:,:,0], cmap='gray')
ax3.imshow(conv2d_result[0,:,:,1], cmap='gray')
ax4.imshow(conv2d_result[0,:,:,2], cmap='gray')

fig.tight_layout()
plt.show()

## **Pooling 연산**

- **특징 맵을 요약하여 공간 크기 감소 시키는 방식**
- CNN 에서 pooling layer는 네트워크의 파라미터 갯수나 연산량을 줄이기 위해 input에서 spatial 하게 downsampling을 진행해 사이즈를 줄이는 역할을 합니다. 일반적으로 CNN에서는 Convolution layer 다음에 들어갑니다.

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as img

fig = plt.figure(figsize=(10,10))  # 가로세로 크기 inch단위

ax1 = fig.add_subplot(1,3,1)
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

ori_image = img.imread('/content/image.png')
ax1.imshow(ori_image)

# 원본 이미지의 shape : (429, 640, 3) => (height, width, color)

# 입력이미지의 형태
# (1, 429, 640, 3) => (이미지 개수, height, width, color)
input_image = ori_image.reshape((1,) + ori_image.shape)
input_image = input_image.astype(np.float32)
print('Convolution input image.shape : {}'.format(input_image.shape))

# 입력이미지 channel 변경
# (1, 429, 640, 1) => (이미지 개수, height, width, color)
# slicing을 이용하여 첫번째 R(Red) 값만 이용
channel_1_input_image = input_image[:,:,:,0:1]
print('Channel 변경 input_image.shape : {}'.format(channel_1_input_image.shape))


# filter
# (3,3,1,1) => (filter height, filter width, filter channel, filter 개수)
# weight = np.random.rand(3,3,1,1)
weight = np.array([[[[-1]],[[0]],[[1]]],
                   [[[-1]],[[0]],[[1]]],
                   [[[-1]],[[0]],[[1]]]])
print('적용할 filter shape : {}'.format(weight.shape))

# stride : 1 (가로1, 세로1)
# padding = 'VALID'

conv2d = tf.nn.conv2d(channel_1_input_image,
                      weight,
                      strides=[1,1,1,1],
                      padding='VALID')

conv2d_result = conv2d.numpy()

print('Convolution 결과 shape : {}'.format(conv2d_result.shape))

# 아래의 코드는 이미지가 1장이기 때문에 필요없다.
# i = np.swapaxes(conv2d_result,0,3)    # (1, 424, 638, 1)
#                                       # (이미지개수, height, weight, filter 수)
#                                       # filter수만큼 loop 돌리기 위해 axes swap

# for filter_idx, t_img in enumerate(i):
#     ax2.imshow(t_img)

t_img = conv2d_result[0,:,:,:]
ax2.imshow(t_img, cmap='gray')

#-----------------
## pooling 처리 ##
#-----------------
# ksize = pooling filter의 크기(avg_pool)
pool = tf.nn.max_pool(conv2d_result,
                      ksize=[1,3,3,1],
                      strides=[1,3,3,1],
                      padding='VALID')

pool_result = pool.numpy()
print('Pooling한 결과 shape : {}'.format(pool_result.shape))

t_img = pool_result[0,:,:,:]
ax3.imshow(t_img, cmap='gray')

fig.tight_layout()
plt.show()

## **MNIST Data Set을 이용한 머신러닝 학습**(DNN)
- regression 방식

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing  import MinMaxScaler
from sklearn.model_selection import train_test_split


# 1. Raw Data Loading
df = pd.read_csv('./train.csv')
# display(df.head())
display(df.shape)

In [ ]:
# 2. 데이터 전처리
# - 결측치 처리
# - 우리 데이터는 연습용 데이터로 결측치가 없음

# - 이상치 처리(outlier 처리: z-score)
# - 이상치도 없어요!

# - 눈으로 데이터를 확인하고 갈겁니다.
# pixel 데이터만 분리
img_data = df.drop(columns=['label'], axis=1, inplace=False).values

# label 데이터만 분리
label_data = df['label']

fig = plt.figure()  # 도화지 준비 : figsize=(10,10)
# subplot이 들어간 list 생성
fig_arr = []
for n in range(10):
    fig_arr.append(fig.add_subplot(2,5,n+1))
    fig_arr[n].imshow(img_data[n].reshape(28,28),
                      cmap='Blues', #gray
                      interpolation='nearest')

plt.tight_layout()  # layout을 알아서 잘 정리해서
plt.show()
# 숫자 이미지가 정상적으로 처리됨을 확인함.

In [ ]:
# 3. Feature Engineering (학습이 적합한 방법으로 데이터를 변형하는 작업: 새로운 변수 추가)
# 이미지인 경우 특별히 해야 할 일이 없음


# 4. 독립면수와 종속변수를 분리
x_data = df.drop('label', axis=1, inplace=False).values
t_data = df['label'].values.reshape(-1,1)    # Series데이터를 2차원 데이터로 만든다. one-hot 인코딩 처리는 keras에게 맡깁니다.


In [ ]:
# 5. Normalization(정규화) : Min-max(0~1), Standardization
scaler = MinMaxScaler()     # scaler 객체 생성
scaler.fit(x_data)          # scaler에게 최대, 최소값을 알려줘서 준비시켜줍니다.
x_data_norm = scaler.transform(x_data)
# x_data_norm = scaler.fit.transform(x_data)

In [ ]:
# 6. 학습데이터와 테스트데이터 분리
x_train, x_test, t_train, t_test = train_test_split(x_data_norm,
                                                    t_data,
                                                    test_size=0.3,
                                                    random_state=42)


# 7. 모델 구현
model = Sequential()
model.add(Flatten(input_shape=(784,)))  # input_layer
model.add(Dense(units=256, activation='relu'))   # hidden_layer
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=10, activation='softmax')) # output_layer

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])       # 평가 기준

model.fit(x_train, t_train,
          epochs=100,
          verbose=1,
          batch_size=100,
          validation_split=0.3)


- **전체 코드**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing  import MinMaxScaler
from sklearn.model_selection import train_test_split


# 1. Raw Data Loading
df = pd.read_csv('./train.csv')
# display(df.head())
display(df.shape)


# 2. 데이터 전처리
# - 결측치 처리
# - 우리 데이터는 연습용 데이터로 결측치가 없음

# - 이상치 처리(outlier 처리: z-score)
# - 이상치도 없어요!

# - 눈으로 데이터를 확인하고 갈겁니다.
# pixel 데이터만 분리
img_data = df.drop(columns=['label'], axis=1, inplace=False).values

# label 데이터만 분리
label_data = df['label']

fig = plt.figure()  # 도화지 준비 : figsize=(10,10)
# subplot이 들어간 list 생성
fig_arr = []
for n in range(10):
    fig_arr.append(fig.add_subplot(2,5,n+1))
    fig_arr[n].imshow(img_data[n].reshape(28,28),
                      cmap='Blues', #gray
                      interpolation='nearest')

plt.tight_layout()  # layout을 알아서 잘 정리해서
plt.show()
# 숫자 이미지가 정상적으로 처리됨을 확인함.


# 3. Feature Engineering (학습이 적합한 방법으로 데이터를 변형하는 작업: 새로운 변수 추가)
# 이미지인 경우 특별히 해야 할 일이 없음


# 4. 독립면수와 종속변수를 분리
x_data = df.drop('label', axis=1, inplace=False).values
t_data = df['label'].values.reshape(-1,1)    # Series데이터를 2차원 데이터로 만든다. one-hot 인코딩 처리는 keras에게 맡깁니다.


# 5. Normalization(정규화) : Min-max(0~1), Standardization
scaler = MinMaxScaler()     # scaler 객체 생성
scaler.fit(x_data)          # scaler에게 최대, 최소값을 알려줘서 준비시켜줍니다.
x_data_norm = scaler.transform(x_data)
# x_data_norm = scaler.fit.transform(x_data)


# 6. 학습데이터와 테스트데이터 분리
x_train, x_test, t_train, t_test = train_test_split(x_data_norm,
                                                    t_data,
                                                    test_size=0.3,
                                                    random_state=42)


# 7. 모델 구현
model = Sequential()
model.add(Flatten(input_shape=(784,)))  # input_layer
model.add(Dense(units=256, activation='relu'))   # hidden_layer
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=10, activation='softmax')) # output_layer

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])       # 평가 기준

model.fit(x_train, t_train,
          epochs=100,
          verbose=1,
          batch_size=100,
          validation_split=0.3)


## **MNIST Data Set을 이용한 CNN 학습**(CNN)
- https://kr.mathworks.com/discovery/convolutional-neural-network.html?utm_source=chatgpt.com

In [ ]:
# 시간이 오래 걸린다.
# MNIST Data Set을 이용한 CNN 학습
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing  import MinMaxScaler
from sklearn.model_selection import train_test_split


# 1. Raw Data Loading
df = pd.read_csv('./train.csv')
# display(df.head())
display(df.shape)


# 2. 데이터 전처리
# - 결측치 처리
# - 우리 데이터는 연습용 데이터로 결측치가 없음

# - 이상치 처리(outlier 처리: z-score)
# - 이상치도 없어요!

# - 눈으로 데이터를 확인하고 갈겁니다.
# pixel 데이터만 분리
img_data = df.drop(columns=['label'], axis=1, inplace=False).values

# label 데이터만 분리
label_data = df['label']

fig = plt.figure()  # 도화지 준비 : figsize=(10,10)
# subplot이 들어간 list 생성
fig_arr = []
for n in range(10):
    fig_arr.append(fig.add_subplot(2,5,n+1))
    fig_arr[n].imshow(img_data[n].reshape(28,28),
                      cmap='gray', #Bules
                      interpolation='nearest')

plt.tight_layout()  # layout을 알아서 잘 정리해서
plt.show()
# 숫자 이미지가 정상적으로 처리됨을 확인함.


# 3. Feature Engineering (학습이 적합한 방법으로 데이터를 변형하는 작업: 새로운 변수 추가)
# 이미지인 경우 특별히 해야 할 일이 없음


# 4. 독립면수와 종속변수를 분리
x_data = df.drop('label', axis=1, inplace=False).values
t_data = df['label'].values.reshape(-1,1)    # Series데이터를 2차원 데이터로 만든다. one-hot 인코딩 처리는 keras에게 맡깁니다.


# 5. Normalization(정규화) : Min-max(0~1), Standardization
scaler = MinMaxScaler()     # scaler 객체 생성
scaler.fit(x_data)          # scaler에게 최대, 최소값을 알려줘서 준비시켜줍니다.
x_data_norm = scaler.transform(x_data)
# x_data_norm = scaler.fit.transform(x_data)


# 6. 학습데이터와 테스트데이터 분리
x_train, x_test, t_train, t_test = train_test_split(x_data_norm,
                                                    t_data,
                                                    test_size=0.3,
                                                    random_state=42)


# 7. 모델 구현(CNN : 4치원)
model = Sequential()

model.add(Conv2D(filters=32,
                 kernel_size=(3,3),
                 strides=(1,1),
                 activation='relu',
                 input_shape=(28,28,1))) # input_shape: 이미지 한개의 사이즈(3차원)
                 # 모델에 첫번째로 추가되는 layer는 input layer 성격을 갖고 있다.

model.add(Conv2D(filters=64,
                 kernel_size=(3,3),
                 strides=(1,1),
                 activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2)))   # 이미지를 반으로 나눈다.

model.add(Conv2D(filters=128,
                 kernel_size=(3,3),
                 strides=(1,1),
                 activation='relu'))
# 필터의 갯수는 앞보다 뒷쪽의 conv layer filter의 갯수가 크다.
# 왜냐하면 뒷쪽은 이미지 크기가 작아지기 때문에 filter의 갯수가 커진다.

model.add(MaxPooling2D(pool_size=(2,2)))   # 이미지를 반으로 나눈다.

# model.summary()   # 모델의 구성을 확인하고 싶을 때 사용
'''
# output shape 설명
(None, 26, 26, 32)
None : 데이터가 아직 들어오지 않았음.
26, 26, 32 : (32:(3x3)필터개수, stride=(1,1)기 때문에 26x26)

#Params 설명:
변수의 갯수
'''
# classification 작업 시작
model.add(Flatten())
# model.summary()

model.add(Dense(units=256,
                activation='relu'))   # hidden_layer
# cnn에서는 hidden layer를 1개 정도로 적게 사용한다.


model.add(Dense(units=10,
                activation='softmax')) # output_layer

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])       # 평가 기준

model.fit(x_train.reshape(-1,28,28,1),
          t_train,
          epochs=100,
          verbose=1,
          batch_size=100,
          validation_split=0.3)





## **결론** :

- 학습을 많이 하면 할 수록 **Filter가 좋아진다**.-> 좋은 모델이 된다.
- 그렇기 때문에 다른 모델에서는 좋아진 Filter 부분만 가져오는 방법을 사용하고 즉, **Feature Extraction 부분을 가져다 사용함**.  이것이 바로 **전이학습(Transfer Learning)**이라고 한다.  --> LesNet GoogleNet 등등 ,


- 좀 더 좋아지기 위해서 기존에 전이학습된 필터는 수정을 안했으나, 좀 더 좋은 성능을 내기 위해 가져온 Feature Extraction 부분 중 **특정 필터(마지막 필터)는 수정**을 하여 나에게 적당한 방법으로 적용하는 방법이 있으며, 이것을 **파인 튜닝**(Fine Tunning)이라고 한다.



- 좋은 Transfer learning model을 사용하려면
    - 내 이미지에 맞는 적합한 모델을 사용하는 것이 중요하다.  